## Сергей Гришаев
## Senior Data Engineer @ SberMarket

In [1]:
from IPython.display import IFrame, Image

## Мотивация создания Apache Spark

### Рассмотрим два примера приложений:
- Обучить модель на больших данных (итеративный алгоритм над фиксированным датасетом)
- Провести ad-hoc анализ данных из двух таблиц (выполнить несколько интерактивных запросов с джойнами и группировками)

## Основные недостатки классического MapReduce

- Постоянное чтение/запись во внешнее хранилище
- Сложный API
- Ограниченное число источников/приемников данных
- MapReduce - это только вычислительный фреймворк

https://arxiv.org/pdf/1209.2191.pdf MapReduce is Good Enough? If All You Have is a Hammer, Throw Away Everything That’s Not a Nail! 

<img src="pics/mapper_2.png" width=700/>

<img src="pics/reducer.png" width=700/>

## Apache Spark - это *быстрая* распределенная вычислительная платформа *общего назначения*

1. **Быстрая** 

    - Кеширование рабочих данных в памяти
    - Ленивые вычисления


2. **Общего назначения** 

    - Можно реализовать любые вычисления (батчевые, итеративные, интерактивные, в режиме реального времени)
    - Более общая модель программирования (узлы в графе вычислений могут быть не только map и reduce)
    - Высокоуровневый API (Scala, Java, Python, R)
    - Локальная установка



<img src="pics/spark_stack.png" width=1000/>

## Множество источников данных

<img src="pics/spark_data_sources.jpg" width=1000/>

## Архитектура Apache Spark

<img src="pics/SparkYARN.png" width=800/>

## Запуск в локальном режиме

<img src="pics/schema_local_2.png" width=600/>

## Запуск PySpark

In [2]:
import os
import sys
import pandas 
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))


## SparkContext (sc) - это основной управляющий объект.

In [137]:
from pyspark import SparkContext, SparkConf

config = SparkConf()
config.set("spark.app.name", "Sergey Grishaev Spark RDD app")

sc = SparkContext(conf=config)

## Для получения всех установленных опций конфигурации можно использовать `sc.getConf()`

In [138]:
sc

<SparkContext master=yarn appName=Sergey Grishaev Spark RDD app>

In [5]:
sc.getConf().getAll()

[('spark.history.kerberos.keytab', 'none'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.ui.proxyBase', '/proxy/application_1673689354371_0045'),
 ('spark.history.fs.cleaner.maxAge', '7d'),
 ('spark.history.ui.port', '18081'),
 ('spark.driver.extraLibraryPath',
  '/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 ('spark.shuffle.io.serverThreads', '128'),
 ('spark.sql.streaming.streamingQueryListeners', ''),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES',
  'http://spark-master-1.newprolab.com:8088/proxy/application_1673689354371_0045'),
 ('spark.executor.extraLibraryPath',
  '/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 ('spark.app.id', 'application_1673689354371_0045'),
 ('spark.jars.repositories', 'https://repos.spark-packages.org/'),
 ('spark.shuffle.file.buffer', '1m'),
 ('spark.yarn.dist.jars',
  'file:///data/home/teacher/.iv

## Основные абстракции Spark
- RDD - Resilient Distributed Dataset - представляет собой распределенную коллекцию данных. RDD обладает свойствами 
    - неизменяемости (операции над RDD приводят к созданию нового RDD) 
    - отказоустойчивости (в случае утери данных на каком либо шаге выполнения данные восстанавливаются из источника) 
    - поддерживает ленивые вычисления (вычисления не производятся если их результат не сохраняется во вне или не используется в других вычислениях)
- DAG - Directed Acyclic Graph - Представляет собой набор вершин и ребер, где вершины представляют собой RDD, а ребра операции над RDD

## Существует два способа создать RDD
- распределить коллекцию объектов с драйвера
- загрузить внешний датасет

## 1. Распределить коллекцию с драйвера

In [7]:
import numpy as np
vocabulary = ("Apache", "Spark", "Hadoop")
numbers = np.random.randint(10, size=10000)
words = np.random.choice(vocabulary, size=10000)
collection = zip(numbers, words)

In [9]:
collection

In [8]:
rdd = sc.parallelize(collection)

In [10]:
rdd.getNumPartitions()

3

In [11]:
rdd.count()

10000

In [38]:
dir(rdd)

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_computeFractionForSampleSize',
 '_defaultReducePartitions',
 '_id',
 '_is_barrier',
 '_jrdd',
 '_jrdd_deserializer',
 '_memory_limit',
 '_pickled',
 '_reserialize',
 '_to_java_object_rdd',
 'aggregate',
 'aggregateByKey',
 'barrier',
 'cache',
 'cartesian',
 'checkpoint',
 'coalesce',
 'cogroup',
 'collect',
 'collectAsMap',
 'combineByKey',
 'context',
 'count',
 'countApprox',
 'countApproxDistinct',
 'countByKey',
 'countByValue',
 'ctx',
 'distinct',
 'filter',
 'first',
 'flatMap',
 'flatMapValues',
 'fold',
 'foldByKey',
 'foreach',
 'foreachPartition',
 'fullOuterJoin',
 'getCheckpo

In [40]:
rdd.toDebugString()

b'(3) ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:195 []'

In [12]:
rdd.count()

10000

In [13]:
rdd.take(10)

[(8, 'Spark'),
 (5, 'Apache'),
 (5, 'Hadoop'),
 (5, 'Spark'),
 (0, 'Spark'),
 (1, 'Apache'),
 (9, 'Spark'),
 (8, 'Hadoop'),
 (8, 'Spark'),
 (3, 'Spark')]

In [14]:
rdd.getNumPartitions()

3

## 2. Загрузить внешний датасет (датасет загружается из HDFS)

In [15]:
!hdfs dfs -ls /lectures/lecture01/data/ips.txt

-rw-r--r--   3 hdfs hdfs      19462 2022-01-06 18:47 /lectures/lecture01/data/ips.txt


In [16]:
!hdfs dfs -cat /lectures/lecture01/data/ips.txt

192.168.0.1	CHINA
192.168.0.2	CHINA
192.168.0.3	CHINA
192.168.0.4	CHINA
192.168.0.5	CHINA
192.168.0.6	CHINA
192.168.0.7	CHINA
192.168.0.8	CHINA
192.168.0.9	CHINA
192.168.0.10	CHINA
192.168.0.11	CHINA
192.168.0.12	CHINA
192.168.0.13	CHINA
192.168.0.14	CHINA
192.168.0.15	CHINA
192.168.0.16	CHINA
192.168.0.17	CHINA
192.168.0.18	CHINA
192.168.0.19	CHINA
192.168.0.20	CHINA
192.168.0.21	CHINA
192.168.0.22	CHINA
192.168.0.23	CHINA
192.168.0.24	CHINA
192.168.0.25	CHINA
192.168.0.26	CHINA
192.168.0.27	CHINA
192.168.0.28	CHINA
192.168.0.29	CHINA
192.168.0.30	CHINA
192.168.0.31	CHINA
192.168.0.32	CHINA
192.168.0.33	CHINA
192.168.0.34	CHINA
192.168.0.35	CHINA
192.168.0.36	CHINA
192.168.0.37	CHINA
192.168.0.38	CHINA
192.168.0.39	CHINA
192.168.0.40	CHINA
192.168.0.41	CHINA
192.168.0.42	CHINA
192.168.0.43	CHINA
192.168.0.44	CHINA
192.168.0.45	CHINA
192.168.0.46	CHINA
192.168.0.47	CHINA
192.168.0.48	CHINA
192.168.0.49	CHINA
192.168.0.50	CHINA
192.168.0.51	CHINA
192.168.0.52	CHINA
192.168.0.53	CHINA
19

In [17]:
rdd2 = sc.textFile("/lectures/lecture01/data/ips.txt")

In [18]:
rdd2.take(10)

['192.168.0.1\tCHINA',
 '192.168.0.2\tCHINA',
 '192.168.0.3\tCHINA',
 '192.168.0.4\tCHINA',
 '192.168.0.5\tCHINA',
 '192.168.0.6\tCHINA',
 '192.168.0.7\tCHINA',
 '192.168.0.8\tCHINA',
 '192.168.0.9\tCHINA',
 '192.168.0.10\tCHINA']

In [19]:
rdd2.count()

1000

In [20]:
rdd2.getNumPartitions()

2

In [30]:
rdd2.repartition(9).saveAsTextFile("ips_repart_v4.txt")

In [34]:
!hadoop dfs -ls /user/teacher/ips_repart_v4.txt


ls: `/data/home/teacher/ips_repart_v4.txt': No such file or directory


In [32]:
rdd3 = sc.textFile("ips_repart_v4.txt")

In [33]:
rdd3.getNumPartitions()

9

## RDD API состоит из операции двух типов:
- action
- transformation

<img src="pics/rdd_action_transformation.png" width=800/>

### Трансформация преобразовывает RDD в другой RDD и не приводит к вычислению графа

In [44]:
rdd = sc.parallelize(range(10000000))
rdd2 = rdd.filter(lambda x: x % 2)

In [42]:
rdd2.getNumPartitions()

3

In [52]:
rdd2.getNumPartitions()

3

In [55]:
rdd3 = rdd2.repartition(8)

In [56]:
rdd3.getNumPartitions()

8

### Action заставляет Spark вычислить граф и вернуть результат либо на драйвер, либо во внешнее хранилище

In [49]:
rdd2.count()

5000000

In [51]:
rdd2.take(10)

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]

### Трансформации можно применять одну за другой, никаких вычислений не будет сделано, пока не будет вызван action

In [57]:
rdd = sc.parallelize(range(1000000))

In [58]:
rdd2 = rdd.filter(lambda x: x % 1000).filter(lambda x: x % 7)
rdd2

PythonRDD[49] at RDD at PythonRDD.scala:53

In [59]:
rdd3 = rdd2.map(lambda x: x * 2)
rdd3

PythonRDD[50] at RDD at PythonRDD.scala:53

In [61]:
d = rdd3.collect()

In [63]:
type(d)

list

### `take()` пытается минимизировать число обращений к партициям, поэтому может возвращать смещенные результаты

In [64]:
rdd.take(10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### Будьте аккуратны с `collect()`, потому что он загружает все данные из RDD на драйвер. Это может легко привести к Out of Memory exception

In [65]:
rdd.collect()[:20]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

### Если нужно получить небольшое число записей на драйвер и, при этом, сохранить распределение, то лучше сделать выборку

In [66]:
rdd.takeSample(withReplacement=False, num=20, seed=5757)

[347399,
 536788,
 94124,
 169558,
 729189,
 836541,
 973939,
 512558,
 171385,
 123303,
 364321,
 587839,
 139749,
 532891,
 757568,
 969298,
 544812,
 690521,
 175014,
 895109]

## Познакомимся с данными. Будем работать с двумя таблицами

![](pics/data_table1.png)

![](pics/data_table2.png)

### Примеры трансформаций

In [67]:
rdd = sc.textFile("/lectures/lecture01/data/ips.txt")

In [68]:
rdd.getNumPartitions()

2

In [69]:
rdd.take(5)

['192.168.0.1\tCHINA',
 '192.168.0.2\tCHINA',
 '192.168.0.3\tCHINA',
 '192.168.0.4\tCHINA',
 '192.168.0.5\tCHINA']

In [70]:
ips = rdd.map(lambda x: x.split("\t"))

In [71]:
ips.take(5)

[['192.168.0.1', 'CHINA'],
 ['192.168.0.2', 'CHINA'],
 ['192.168.0.3', 'CHINA'],
 ['192.168.0.4', 'CHINA'],
 ['192.168.0.5', 'CHINA']]

In [72]:
ips_filtered = ips.filter(lambda x: x[1] != "CHINA")

In [73]:
ips_filtered.take(5)

[['192.168.0.201', 'RUSSIA'],
 ['192.168.0.202', 'RUSSIA'],
 ['192.168.0.203', 'RUSSIA'],
 ['192.168.0.204', 'RUSSIA'],
 ['192.168.0.205', 'RUSSIA']]

In [74]:
!hdfs dfs -ls /lectures/lecture01/data/log.txt

Found 5 items
-rw-r--r--   3 hdfs hdfs          0 2022-01-06 18:47 /lectures/lecture01/data/log.txt/_SUCCESS
-rw-r--r--   3 hdfs hdfs   12650786 2022-01-06 18:47 /lectures/lecture01/data/log.txt/part-00000
-rw-r--r--   3 hdfs hdfs   12644978 2022-01-06 18:47 /lectures/lecture01/data/log.txt/part-00001
-rw-r--r--   3 hdfs hdfs   12646744 2022-01-06 18:47 /lectures/lecture01/data/log.txt/part-00002
-rw-r--r--   3 hdfs hdfs   12676672 2022-01-06 18:47 /lectures/lecture01/data/log.txt/part-00003


In [75]:
raw_logs = sc.textFile("/lectures/lecture01/data/log.txt")

In [76]:
raw_logs.getNumPartitions()

4

In [77]:
raw_logs.take(5)

['192.168.0.10\tERROR\tWhen production fails in dipsair, whom you gonna call?',
 '192.168.0.39\tINFO\tJust an info message passing by',
 '192.168.0.35\tINFO\tJust an info message passing by',
 '192.168.0.19\tINFO\tJust an info message passing by',
 '192.168.0.23\tERROR\tWhen production fails in dipsair, whom you gonna call?']

In [78]:
logs = raw_logs.map(lambda x: x.split("\t"))

In [79]:
logs.take(5)

[['192.168.0.10',
  'ERROR',
  'When production fails in dipsair, whom you gonna call?'],
 ['192.168.0.39', 'INFO', 'Just an info message passing by'],
 ['192.168.0.35', 'INFO', 'Just an info message passing by'],
 ['192.168.0.19', 'INFO', 'Just an info message passing by'],
 ['192.168.0.23',
  'ERROR',
  'When production fails in dipsair, whom you gonna call?']]

In [80]:
logs.flatMap(lambda x: x[2].split()).take(20)

['When',
 'production',
 'fails',
 'in',
 'dipsair,',
 'whom',
 'you',
 'gonna',
 'call?',
 'Just',
 'an',
 'info',
 'message',
 'passing',
 'by',
 'Just',
 'an',
 'info',
 'message',
 'passing']

In [81]:
words = logs.flatMap(lambda x: x[2].split())

In [83]:
words.groupBy(lambda x: x).count()

TypeError: groupBy() missing 1 required positional argument: 'f'

## Зачем нужны отдельные трансформации и отдельные action?

![](pics/dag1.png)

![](pics/dag2.png)

### Последовательность трансформаций определяет граф вычислений (DAG - direct acyclic graph). В нем есть партиции и зависимости между партициями. Таким образом Spark имеет всю необходимую информацию для вычилсения графа в любой точке и возможных оптимизаций

![](pics/dag3.png)

### Трансформации бывают *узкими*

![](pics/narrow_transformation.png)

### И *широкими*

![](pics/wide_transformation.png)

### Широкие трансформации разделяют джоб на стейджи. Между стейджами происходит shuffle данных, которого надо избегать. Кстати почему?

## Summary:
    
    - Каждое действие (action) инициирует новое задание (job)
    - Spark анализирует граф RDD и строит план выполнения
    - План выполнения может включать несколько этапов (stages)
    - Каждый этап состоит из набора задач (tasks) - полняется один и тот же код на разных кусочках данных

In [86]:
rdd = sc.textFile("/lectures/lecture01/data/ips.txt")
ips = rdd.map(lambda x: x.split("\t"))
ips_filtered = ips.filter(lambda x: x[1] != "CHINA")
ips_filtered.take(5)
ips_filtered.toDebugString()

b'(2) PythonRDD[80] at RDD at PythonRDD.scala:53 []\n |  /lectures/lecture01/data/ips.txt MapPartitionsRDD[78] at textFile at NativeMethodAccessorImpl.java:0 []\n |  /lectures/lecture01/data/ips.txt HadoopRDD[77] at textFile at NativeMethodAccessorImpl.java:0 []'

In [87]:
raw_logs = sc.textFile("/lectures/lecture01/data/log.txt")
logs = raw_logs.map(lambda x: x.split("\t"))
words = logs.flatMap(lambda x: x[2].split())
words.groupBy(lambda x: x).count()

19

## Персистентность и кэширование

### RDD вычисляются лениво, когда вызывается action. Часто мы хотим вызвать несколько actions для одного и тоге же RDD. Если мы просто сделаем это, то граф будет полностью перевычисляться каждый раз.

In [88]:
ips.count()

1000

In [89]:
ips.top(10)

[['192.168.3.99', 'USA'],
 ['192.168.3.98', 'USA'],
 ['192.168.3.97', 'USA'],
 ['192.168.3.96', 'USA'],
 ['192.168.3.95', 'USA'],
 ['192.168.3.94', 'USA'],
 ['192.168.3.93', 'USA'],
 ['192.168.3.92', 'USA'],
 ['192.168.3.91', 'USA'],
 ['192.168.3.90', 'USA']]

### Чтобы этого избежать, мы можем закэшировать RDD в памяти. Кэширование произойдет при вызове первого action.

In [90]:
ips_cached = ips.cache()

In [91]:
ips_cached.count()

1000

In [92]:
ips_cached.top(20)

[['192.168.3.99', 'USA'],
 ['192.168.3.98', 'USA'],
 ['192.168.3.97', 'USA'],
 ['192.168.3.96', 'USA'],
 ['192.168.3.95', 'USA'],
 ['192.168.3.94', 'USA'],
 ['192.168.3.93', 'USA'],
 ['192.168.3.92', 'USA'],
 ['192.168.3.91', 'USA'],
 ['192.168.3.90', 'USA'],
 ['192.168.3.9', 'USA'],
 ['192.168.3.89', 'USA'],
 ['192.168.3.88', 'USA'],
 ['192.168.3.87', 'USA'],
 ['192.168.3.86', 'USA'],
 ['192.168.3.85', 'USA'],
 ['192.168.3.84', 'USA'],
 ['192.168.3.83', 'USA'],
 ['192.168.3.82', 'USA'],
 ['192.168.3.81', 'USA']]

### `cache()` сохраняет RDD в памяти. Для большего контроля можно использовать `persist(storage_level)`:
+ MEMORY_ONLY
+ MEMORY_AND_DISK
+ DISK_ONLY
+ MEMORY_ONLY_2
+ MEMORY_AND_DISK_2

## Удаление данных из кэша
 - Автоматическое (LRU cache)

In [93]:
ips_cached.unpersist()

PythonRDD[90] at RDD at PythonRDD.scala:53

### Обработка отказов

- Промежуточные результаты не сохраняются в HDFS
- RDD по-умолчанию не реплицируются
- Для восстановления потерянных данных используется информация о происхождении
- Spark восстанавливает потерянные фрагменты RDD, заново вычисляя их путем применения трансформаций

## PairRDD (ключ-значение)

### PairRDD - это RDD для работы с парами ключ-значение. Spark предполагает, что PairRDD содержит в себе объекты, состящие ровно из двух элементов! PairRDD предоставляют методы группировки, аггрегации и объединения (join) двух RDD

### Пусть есть задача подсчитать распределение кодов ERROR и WARNING в лог-файле

In [94]:
raw_logs.take(5)

['192.168.0.10\tERROR\tWhen production fails in dipsair, whom you gonna call?',
 '192.168.0.39\tINFO\tJust an info message passing by',
 '192.168.0.35\tINFO\tJust an info message passing by',
 '192.168.0.19\tINFO\tJust an info message passing by',
 '192.168.0.23\tERROR\tWhen production fails in dipsair, whom you gonna call?']

In [95]:
(raw_logs.filter(lambda x: "INFO" not in x)
         .map(lambda x: (x.split("\t")[1], 1))\
         .groupByKey()
         .collect())

[('ERROR', <pyspark.resultiterable.ResultIterable at 0x7f7b9acb6c50>),
 ('WARNING', <pyspark.resultiterable.ResultIterable at 0x7f7b9acb6860>)]

In [96]:
(raw_logs.filter(lambda x: "INFO" not in x)
         .map(lambda x: (x.split("\t")[1], 1))\
         .groupByKey()
         .map(lambda x: (x[0], len(x[1])))
         .collect())

[('WARNING', 250004), ('ERROR', 49948)]

### Или немного проще

In [97]:
(raw_logs.filter(lambda x: "INFO" not in x)
         .map(lambda x: (x.split("\t")[1], 1))
         .countByKey()
         .items())

dict_items([('ERROR', 49948), ('WARNING', 250004)])

### Стоит заметить, что `groupByKey()` предполагает перемещение всех записей с одним ключом на один экзекьютор. В случае очень скошенных распределений это может привести к падению экзекьютора с OOM. Поэтому всегда при группировках стоит подумать об использовании `reduceByKey()`.

In [98]:
def plus(x, y):
    return x + y

In [99]:
(raw_logs.filter(lambda x: "INFO" not in x)
         .map(lambda x: (x.split("\t")[1], 1))\
         .reduceByKey(plus)
         .collect())

[('WARNING', 250004), ('ERROR', 49948)]

In [100]:
(raw_logs.filter(lambda x: "INFO" not in x)
         .map(lambda x: (x.split("\t")[1], 1))\
         .reduceByKey(lambda x, y: x + y)
         .collect())

[('WARNING', 250004), ('ERROR', 49948)]

## Join

### Два PairRDD можно объединить по ключу
### Поддерживаются inner join, left outer join, right outer join и full outer join

In [101]:
logs.take(5)

[['192.168.0.10',
  'ERROR',
  'When production fails in dipsair, whom you gonna call?'],
 ['192.168.0.39', 'INFO', 'Just an info message passing by'],
 ['192.168.0.35', 'INFO', 'Just an info message passing by'],
 ['192.168.0.19', 'INFO', 'Just an info message passing by'],
 ['192.168.0.23',
  'ERROR',
  'When production fails in dipsair, whom you gonna call?']]

In [102]:
ips.take(5)

[['192.168.0.1', 'CHINA'],
 ['192.168.0.2', 'CHINA'],
 ['192.168.0.3', 'CHINA'],
 ['192.168.0.4', 'CHINA'],
 ['192.168.0.5', 'CHINA']]

In [108]:
type(ips)

pyspark.rdd.PipelinedRDD

In [103]:
logs.join(ips).take(5)

[('192.168.0.64', ('WARNING', 'CHINA')),
 ('192.168.0.64', ('INFO', 'CHINA')),
 ('192.168.0.64', ('INFO', 'CHINA')),
 ('192.168.0.64', ('WARNING', 'CHINA')),
 ('192.168.0.64', ('WARNING', 'CHINA'))]

![](pics/Jackie-Chan-WTF.jpg)

### Не стоит забывать, что Spark предполагает, что PairRDD состоит ровно! из двух элементов, поэтому все остальные элементы просто отбрасываются!

In [104]:
def split_logs(line):
    split = line.split("\t")
    return split[0], split[1:]

In [105]:
logs_cached = raw_logs.map(split_logs).cache()

In [106]:
logs_cached.take(5)

[('192.168.0.10',
  ['ERROR', 'When production fails in dipsair, whom you gonna call?']),
 ('192.168.0.39', ['INFO', 'Just an info message passing by']),
 ('192.168.0.35', ['INFO', 'Just an info message passing by']),
 ('192.168.0.19', ['INFO', 'Just an info message passing by']),
 ('192.168.0.23',
  ['ERROR', 'When production fails in dipsair, whom you gonna call?'])]

In [107]:
logs_cached.join(ips).take(5)

[('192.168.0.23',
  (['ERROR', 'When production fails in dipsair, whom you gonna call?'],
   'CHINA')),
 ('192.168.0.23', (['INFO', 'Just an info message passing by'], 'CHINA')),
 ('192.168.0.23', (['INFO', 'Just an info message passing by'], 'CHINA')),
 ('192.168.0.23',
  (['ERROR', 'When production fails in dipsair, whom you gonna call?'],
   'CHINA')),
 ('192.168.0.23', (['INFO', 'Just an info message passing by'], 'CHINA'))]

## Управление параллелизмом.

### Вспомним, что атомарным уровнем параллелизма в Spark является партиция. Об этом всегда стоит помнить, когда есть проблемы с производительностью приложения

In [109]:
logs.getNumPartitions()

4

### Метод `repartition()` может быть использован для изменения числа партиций.

In [113]:
logs = logs.repartition(1000)

In [115]:
logs.getNumPartitions()

1000

### `repartition()` всегда приводит к равномерному перераспределению данных, что ведет к shuffle. Если Вы уменьшаете число партиций, то стоит использовать `coalesce()`, который может избежать shuffle

In [116]:
logs = logs.coalesce(6)

In [121]:
logs.getNumPartitions()

3

In [120]:
logs = logs.coalesce(3)

In [122]:
logs = logs.coalesce(6)

In [123]:
logs.getNumPartitions()

3

### Узнать дефолтный уровень параллелизма можно из конфига. По-умолчанию, при работе с YARN, использукется общее число ядер, выделенных этому SparkContext на всех экзекьюторах, либо 2. Что больше.

In [124]:
print(sc.getConf().get("spark.default.parallelism"))

None


In [125]:
sc.parallelize(range(100000)).getNumPartitions()

3

## Broadcast

### Broadcast-объект - это неизменяемая переменная, которая разделяется между всеми экзекьюторами
### Дистрибуция broadcast-объекта производится быстро и эффективно p2p-протоколом

### Реализуем map-side join с помощью broadcast-объекта

In [126]:
ips_local = dict(ips.collect())

In [127]:
ips_local['192.168.0.10']

'CHINA'

In [128]:
ips_broadcasted = sc.broadcast(ips_local)

In [130]:
ips_broadcasted.value['192.168.0.10']

'CHINA'

In [129]:
type(ips_broadcasted.value)

dict

In [131]:
logs_cached.take(5)

[('192.168.0.10',
  ['ERROR', 'When production fails in dipsair, whom you gonna call?']),
 ('192.168.0.39', ['INFO', 'Just an info message passing by']),
 ('192.168.0.35', ['INFO', 'Just an info message passing by']),
 ('192.168.0.19', ['INFO', 'Just an info message passing by']),
 ('192.168.0.23',
  ['ERROR', 'When production fails in dipsair, whom you gonna call?'])]

In [134]:
def resolve_ip(row):
    return ips_broadcasted.value[row[0]], row[0:] ## row[0] is the IP address

In [135]:
logs_cached.map(resolve_ip).take(10)

[('CHINA',
  ('192.168.0.10',
   ['ERROR', 'When production fails in dipsair, whom you gonna call?'])),
 ('CHINA', ('192.168.0.39', ['INFO', 'Just an info message passing by'])),
 ('CHINA', ('192.168.0.35', ['INFO', 'Just an info message passing by'])),
 ('CHINA', ('192.168.0.19', ['INFO', 'Just an info message passing by'])),
 ('CHINA',
  ('192.168.0.23',
   ['ERROR', 'When production fails in dipsair, whom you gonna call?'])),
 ('CHINA',
  ('192.168.0.90',
   ['ERROR', 'When production fails in dipsair, whom you gonna call?'])),
 ('CHINA', ('192.168.0.1', ['INFO', 'Just an info message passing by'])),
 ('CHINA', ('192.168.0.82', ['WARNING', 'Something bad could happen'])),
 ('CHINA', ('192.168.0.89', ['INFO', 'Just an info message passing by'])),
 ('CHINA', ('192.168.0.2', ['INFO', 'Just an info message passing by']))]

## Не забудьте погасить SparkContext!

In [136]:
sc.stop()

#### Полезные ссылки
####  https://spark.apache.org/docs/2.4.7/rdd-programming-guide.html

    